In [1]:
import pickle
with open('cache/df.pickle', mode='rb') as h:
    _, df_test = pickle.load(h)
with open('cache/hex.pickle', mode='rb') as h:
    hex_data = pickle.load(h)

In [2]:
from collections import Counter
labels = df_test['label']
label_counter = Counter(labels)  # dict<label, #_of_occurrence>
id_hls = map(set, hex_data['id_hierarchical_labels'])  # id_hls for id_hierarchical_labels
id_fh = map(lambda x: max(x, key=len), id_hls)  # id_fh for id_full_hierarchy

In [3]:
import numpy as np
def get_accuracy(Y, normalize=True):
    """
    Tyoe 0: Leaf accuracy, used in Deng et al, p13
    Type 1: Is the full hierarchy
    Type 2: Covers the full hierarchy
    Type 3: Is any partial hierarchy
    Type 4: Covers any partial hierarchy
    Implication: 1->2, 1->3, 2->4, 3->4
    Note that type 2 and 3 are exclusive
    Args:
        Y: N*D matrix
    Returns:
        21*5 matrix, where the last row is the overall accuracy
    """
    by_labels = np.zeros((20, 5), dtype=np.float32)
    for i in range(0, len(Y)):
        l = labels[i]
        y = Y[i]
        y = (y / y.max()) > threshold  # TODO: is normalization necessary?
        if y[l]:
            by_labels[l, 0] += 1  # TODO: how is leaf accuracy defined???
        y = np.nonzero(y)[0]
        if tuple(y) == id_fh[l]:
            by_labels[l, 1:] += 1
        elif set(id_fh[l]) in set(y):
            by_labels[l, [2, 4]] += 1
        elif tuple(y) in id_hls[l]:
            by_labels[l, [3, 4]] += 1
        elif any(set(x) in set(y) for x in id_hls[l]):
            by_labels[l, 4] += 1
    overall = np.sum(by_labels, axis=0, dtype=np.float32)
    if normalize:
        for i in range(0, 20):
            by_labels[i] /= label_counter[i]
        overall /= len(Y)
    else:
        by_labels = by_labels.astype(np.uint32)
        overall = overall.astype(np.uint32)
    return np.vstack((by_labels, overall))

In [4]:
np.set_printoptions(precision=4)
id_name = hex_data['id_name']
def accurracy_str(accuracy):
    lines = str(accuracy).split('\n')
    lines[0] = lines[0].replace('[[', '[')
    lines[-1] = lines[-1].replace(']]', ']')
    for i in range(0, 20):
        lines[i] = '{} {}'.format(lines[i].replace(' [', '['), id_name[i])
    lines[-1] = '{} {}'.format(lines[-1].replace(' [', '['), 'overall')
    return '\n'.join(lines)

In [5]:
# caffe accuracy
threshold = 0.2
iter_Y = np.load('../test_caffe_3.npy')
iters = np.linspace(5000, 50000, 10, dtype=np.uint16)
for x in zip(iters, map(get_accuracy, iter_Y)):
    print 'iteration={}'.format(x[0])
    print accurracy_str(x[1])

iteration=5000
[ 0.      0.      0.      0.      0.    ] diningtable
[ 0.      0.      0.      0.      0.    ] chair
[ 0.038   0.      0.      0.0127  0.0127] sofa
[ 0.      0.      0.      0.      0.    ] bottle
[ 0.      0.      0.      0.      0.    ] pottedplant
[ 0.1702  0.      0.      0.      0.    ] tvmonitor
[ 0.1359  0.068   0.068   0.3883  0.3883] train
[ 0.2179  0.1923  0.1923  0.5897  0.5897] bus
[ 0.2679  0.2589  0.2589  0.5536  0.5536] car
[ 0.      0.      0.      0.25    0.25  ] bicycle
[ 0.      0.      0.      0.2963  0.2963] motorbike
[ 0.6716  0.0448  0.0448  0.0448  0.0448] aeroplane
[ 0.1392  0.      0.      0.      0.    ] boat
[ 0.      0.      0.      0.      0.    ] cow
[ 0.      0.      0.      0.      0.    ] horse
[ 0.      0.      0.      0.      0.    ] sheep
[ 0.      0.      0.      0.      0.    ] dog
[ 0.0113  0.      0.      0.0056  0.0056] cat
[ 0.2593  0.0074  0.0074  0.0074  0.0074] bird
[ 0.523   0.      0.      0.      0.    ] person
[ 0.1787  

In [6]:
# svm accuracy
threshold = 0
kernel_Y = np.load('../test_svm.npy')
kernels = ['linear', 'poly', 'rbf']
for x in zip(kernels, map(get_accuracy, kernel_Y)):
    print 'kernel={}'.format(x[0])
    print accurracy_str(x[1])

kernel=linear
[ 0.381   0.1667  0.1667  0.5     0.5   ] diningtable
[ 0.1364  0.0909  0.0909  0.4091  0.4091] chair
[ 0.1899  0.1266  0.1266  0.519   0.519 ] sofa
[ 0.1613  0.0968  0.0968  0.2581  0.2581] bottle
[ 0.28    0.16    0.16    0.36    0.36  ] pottedplant
[ 0.5532  0.4255  0.4255  0.6383  0.6383] tvmonitor
[ 0.534   0.4369  0.4369  0.8155  0.8155] train
[ 0.6026  0.5513  0.5513  0.859   0.859 ] bus
[ 0.4643  0.3571  0.3571  0.7589  0.7589] car
[ 0.3654  0.2308  0.2308  0.5577  0.5577] bicycle
[ 0.5185  0.4074  0.4074  0.6296  0.6296] motorbike
[ 0.597   0.5299  0.5299  0.8284  0.8284] aeroplane
[ 0.5316  0.4051  0.4051  0.5949  0.5949] boat
[ 0.2653  0.0612  0.0612  0.5102  0.5102] cow
[ 0.3333  0.2639  0.2639  0.6528  0.6528] horse
[ 0.3514  0.1892  0.1892  0.5135  0.5135] sheep
[ 0.4625  0.2625  0.2625  0.6375  0.6375] dog
[ 0.4689  0.3729  0.3729  0.7571  0.7571] cat
[ 0.5259  0.3704  0.3704  0.6963  0.6963] bird
[ 0.5632  0.3621  0.3621  0.5575  0.5575] person
[ 0.4596  0

In [7]:
state_space = hex_data['state_space']
def to_crf(X):
    def to_crf_step(x):
        scores = map(lambda s: x[s].sum(), state_space)
        return state_space[np.argmax(scores)]
    return np.array(map(to_crf_step, X), dtype=np.bool)

In [8]:
# crf accuracy
threshold = 0
kernel_Y_crf = np.array(map(to_crf, kernel_Y), dtype=np.bool)
kernels = ['linear', 'poly', 'rbf']
for x in zip(kernels, map(get_accuracy, kernel_Y_crf)):
    print 'kernel={}'.format(x[0])
    print accurracy_str(x[1])

kernel=linear
[ 0.2381  0.2381  0.2381  0.7857  0.7857] diningtable
[ 0.0909  0.0909  0.0909  0.5455  0.5455] chair
[ 0.1266  0.1266  0.1266  0.6835  0.6835] sofa
[ 0.129   0.129   0.129   0.4839  0.4839] bottle
[ 0.2     0.2     0.2     0.56    0.56  ] pottedplant
[ 0.4894  0.4894  0.4894  0.766   0.766 ] tvmonitor
[ 0.4757  0.4757  0.4757  0.9223  0.9223] train
[ 0.5897  0.5897  0.5897  0.8974  0.8974] bus
[ 0.4107  0.4107  0.4107  0.875   0.875 ] car
[ 0.3269  0.3269  0.3269  0.7885  0.7885] bicycle
[ 0.4259  0.4259  0.4259  0.7778  0.7778] motorbike
[ 0.5746  0.5746  0.5746  0.8955  0.8955] aeroplane
[ 0.481   0.481   0.481   0.7342  0.7342] boat
[ 0.0816  0.0816  0.0816  0.5918  0.5918] cow
[ 0.2917  0.2917  0.2917  0.75    0.75  ] horse
[ 0.2973  0.2973  0.2973  0.7297  0.7297] sheep
[ 0.3875  0.3875  0.3875  0.8375  0.8375] dog
[ 0.4407  0.4407  0.4407  0.8927  0.8927] cat
[ 0.4519  0.4519  0.4519  0.8889  0.8889] bird
[ 0.477   0.477   0.477   0.7241  0.7241] person
[ 0.399   0